In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from App.logistic_regressor import LogisticRegressor

In [2]:
train=pd.read_csv('./Data/titanic_train.csv',index_col=0)
test=pd.read_csv('./Data/titanic_test.csv',index_col=0)
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
83,1,3,"McDermott, Miss. Brigdet Delia",female,NaN,0,0,330932,7.7875,NaN,Q
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
866,1,2,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,236852,13.0000,NaN,S
399,0,2,"Pain, Dr. Alfred",male,23.0,0,0,244278,10.5000,NaN,S
474,1,2,"Jerwan, Mrs. Amin S (Marie Marthe Thuillard)",female,23.0,0,0,SC/AH Basle 541,13.7917,D,C


In [3]:
test.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
268,1,3,"Persson, Mr. Ernst Ulrik",male,25.0,1,0,347083,7.775,NaN,S
222,0,2,"Bracken, Mr. James H",male,27.0,0,0,220367,13.000,NaN,S
840,1,1,"Marechal, Mr. Pierre",male,NaN,0,0,11774,29.700,C47,C
142,1,3,"Nysten, Miss. Anna Sofia",female,22.0,0,0,347081,7.750,NaN,S
574,1,3,"Kelly, Miss. Mary",female,NaN,0,0,14312,7.750,NaN,Q


In [4]:
print(train.shape)
train.isnull().sum()

(623, 11)


Survived      0
Pclass        0
Name          0
Sex           0
Age         120
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       476
Embarked      1
dtype: int64

In [5]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Survived
0,1,0.642857
1,2,0.479675
2,3,0.242775


In [6]:
train[['Pclass', 'Age']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Age
0,1,40.113382
1,2,30.709469
2,3,25.549882


In [7]:
Pclass_means = train.groupby('Pclass')['Age'].mean()
Pclass_list = [1,2,3]
def age_missing_replace(means, data_frame, Pclass_list):
    for pclass in Pclass_list:
        temp = data_frame['Pclass'] == pclass
        data_frame.loc[temp, 'Age'] = data_frame.loc[temp, 'Age'].fillna(means[pclass]) 
    age_missing_replace(Pclass_means,train,Pclass_list)
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
83,1,3,"McDermott, Miss. Brigdet Delia",female,NaN,0,0,330932,7.7875,NaN,Q
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
866,1,2,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,236852,13.0000,NaN,S
399,0,2,"Pain, Dr. Alfred",male,23.0,0,0,244278,10.5000,NaN,S
474,1,2,"Jerwan, Mrs. Amin S (Marie Marthe Thuillard)",female,23.0,0,0,SC/AH Basle 541,13.7917,D,C


In [8]:
train.Sex.value_counts(dropna=False)

male      410
female    213
Name: Sex, dtype: int64

In [9]:
def assignDeckValue(CabinCode):
    if pd.isnull(CabinCode):
        category = 'Unknown'
    else:
        category = CabinCode[0]
    return category
  
Deck = np.array([assignDeckValue(cabin) for cabin in train['Cabin'].values])

train = train.assign(Deck = Deck)
train.head()


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
PassengerId,,,,,,,,,,,,
83,1,3,"McDermott, Miss. Brigdet Delia",female,NaN,0,0,330932,7.7875,NaN,Q,Unknown
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Unknown
866,1,2,"Bystrom, Mrs. (Karolina)",female,42.0,0,0,236852,13.0000,NaN,S,Unknown
399,0,2,"Pain, Dr. Alfred",male,23.0,0,0,244278,10.5000,NaN,S,Unknown
474,1,2,"Jerwan, Mrs. Amin S (Marie Marthe Thuillard)",female,23.0,0,0,SC/AH Basle 541,13.7917,D,C,D


In [10]:
train[['Deck', 'Survived']].groupby(['Deck'], as_index=False).mean()

,Deck,Survived
0,A,0.500000
1,B,0.705882
2,C,0.631579
3,D,0.739130
4,E,0.750000
5,F,0.636364
6,G,0.500000
7,T,0.000000
8,Unknown,0.302521


In [11]:
def weak_ppl(age):    
    if age<=16:
        return 0
    elif age<=64:
        return 1
    else:
        return 2



In [12]:
train['Age'] = train['Age'].apply(weak_ppl)
train[['Age', 'Survived']].groupby(['Age'], as_index=False).mean()


,Age,Survived
0,0,0.550725
1,1,0.388235
2,2,0.302326


In [13]:
train.drop(['Ticket', 'Cabin'],axis = 1, inplace =True)
test.drop(['Ticket', 'Cabin'],axis = 1, inplace =True)

In [14]:
train.drop(['Deck'],axis=1, inplace=True)
train.SibSp.fillna(0, inplace=True)
train.Parch.fillna(0,inplace=True)
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,,
83,1,3,"McDermott, Miss. Brigdet Delia",female,2,0,0,7.7875,Q
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,1,0,2,11.1333,S
866,1,2,"Bystrom, Mrs. (Karolina)",female,1,0,0,13.0000,S
399,0,2,"Pain, Dr. Alfred",male,1,0,0,10.5000,S
474,1,2,"Jerwan, Mrs. Amin S (Marie Marthe Thuillard)",female,1,0,0,13.7917,C


In [15]:
train['family'] = train['SibSp']+train['Parch']
test['family'] = test['SibSp']+test['Parch'] 
test.drop(['Parch', 'SibSp'],axis =1, inplace=True)
train.drop(['Parch', 'SibSp'],axis =1, inplace=True)


In [16]:
train[['family', 'Survived']].groupby(['family'], as_index=False).mean()

,family,Survived
0,0,0.293194
1,1,0.594828
2,2,0.640625
3,3,0.722222
4,4,0.166667
5,5,0.066667
6,6,0.500000
7,7,0.000000
8,10,0.000000


In [17]:
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean()

,Sex,Survived
0,female,0.755869
1,male,0.197561


In [18]:
train['Title'] = train.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [19]:
train['Title'] = train['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Sir', 'Jonkheer', 'Lady', 'Capt', 'Don'], 'Others')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
train['Title'] = train['Title'].replace('Mlle', 'Miss')

In [20]:
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.571429
1,Miss,0.719298
2,Mr,0.168478
3,Mrs,0.791667
4,Others,0.352941


In [21]:
train.drop(['Name'],axis=1, inplace=True)
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,family,Title
PassengerId,,,,,,,,
83,1,3,female,2,7.7875,Q,0,Miss
9,1,3,female,1,11.1333,S,2,Mrs
866,1,2,female,1,13.0000,S,0,Mrs
399,0,2,male,1,10.5000,S,0,Others
474,1,2,female,1,13.7917,C,0,Mrs


In [22]:
train.Embarked.value_counts(dropna=False)

S      450
C      119
Q       53
NaN      1
Name: Embarked, dtype: int64

In [23]:
train[['Pclass', 'Fare']].groupby(['Pclass'], as_index=False).mean()

,Pclass,Fare
0,1,83.390694
1,2,20.165414
2,3,13.314244


In [24]:
train.Pclass.value_counts(dropna=False)

3    346
1    154
2    123
Name: Pclass, dtype: int64

In [25]:
train.head()


,Survived,Pclass,Sex,Age,Fare,Embarked,family,Title
PassengerId,,,,,,,,
83,1,3,female,2,7.7875,Q,0,Miss
9,1,3,female,1,11.1333,S,2,Mrs
866,1,2,female,1,13.0000,S,0,Mrs
399,0,2,male,1,10.5000,S,0,Others
474,1,2,female,1,13.7917,C,0,Mrs


In [26]:
train.Fare.fillna(13.314,inplace=True)
test.Fare.fillna(13.314,inplace=True)


train.Embarked.fillna('S', inplace=True)
test.Embarked.fillna('S',inplace = True)

In [27]:
dummies_train= []
dummies_test = []

cols = ['Pclass', 'Embarked','Sex','Age','Title']
for col in cols:
    dummies_train.append(pd.get_dummies(train[col]))
    
dummies_training = pd.concat(dummies_train, axis=1)

train = pd.concat((train,dummies_training), axis=1)

train.drop(cols, axis=1, inplace=True)


In [28]:
train.head()

,Survived,Fare,family,1,2,3,C,Q,S,female,male,0,1,2,Master,Miss,Mr,Mrs,Others
PassengerId,,,,,,,,,,,,,,,,,,,
83,1,7.7875,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0
9,1,11.1333,2,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0
866,1,13.0000,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0
399,0,10.5000,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1
474,1,13.7917,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0
